# DaVinci Endowrist control

In this section we will design a complete Endowrist control based on Arduino

<img src="./Images/5_endowrist1.png">

## Arduino serial node

An arduino board is used to interface the sensors and actuators needed to control the Endowrist with the pen-based user interface device:
- read the IMU sensor to obtain the rpw orientation to TCP Endowrist
- read the first push buttom to control the movement in the x direction of Endowrist
- read the second push buttom to open or close the Endowrist gripper
- write to the 3 servomotors the rpw angles to obtain in Endowrist gripper the same orientation as the pen

A custom C++ file is written in arduino board to perform these operations and publish the corresponding topics:
- /pen_orientation: Float32MultiArray message obtained from the IMU rpw angles
- /pen_position: Boolean message obtained from the first pen push buttom
- /pen_open: Boolean message obtained from the second pen push buttom

First of all you have to install rosserial and the arduino libraries

In [ ]:
sudo apt-get install ros-melodic-rosserial ros-melodic-rosserial-server ros-melodic-rosserial-arduino

In [ ]:
cd ~/Arduino/libraries
rm -rf ros_lib
rosrun rosserial_arduino make_libraries.py .

Now you are ready to develop ROS programs adapted to Arduino platform

You will need to install the USB port and give user permissions. Instructions are given in:https://github.com/manelpuig/rubot_hw_ws/blob/master/Documentation/01_Arduino_install.ipynb

The program designed for Arduino is "endowrist_control_ros.ino" located in "Documentation/Files/Arduino_programs"

## Endowrist Pen control node
We have designed a /pen_control node with:
- Subscriber to /pen_orientation: obtained from the IMU rpw angles
- Subscriber to /pen_position: obtained from the first pen push buttom
- Subscriber to /pen_open: obtained from the second pen push buttom
- Subscriber to /pose_current: obtained from the current pose published by /ur5e_node
- Publisher to /pose_target: defining the movement across x axis of endowrist frame

We have created a new python file "endowrist_pen_control.py" in the same ur5control package

A new "endowrist_control.launch" file is created to:
- launch the Arduino serial node
- launch the Endowrist pen control node
- launch the ur5e bringup
- launch the ur5e control node

In [ ]:
<launch>
  <!-- Launch Arduino serial node -->
  <node name="serial_node"        pkg="rosserial_python"      type="serial_node.py">
    <param name="port"              type="string"               value="/dev/ttyACM0"/>
    <param name="baud"              type="int"                  value="57600"/>
  </node>
  <!-- Launch Endowrist pen control node -->
  <node name="endowrist_pen_control" pkg="ur5control" type="endowrist_pen_control.py" output="screen">
  </node>
  <!-- Launch ur5e bringup -->
  <include file="$(find ur5control)/launch/ur5e_bringup_sim.launch"/>
  <!-- Launch ur5e node -->
  <node name="ur5e_node" pkg="ur5control" type="ur5e_pose2robot.py" output="screen">
  </node>
</launch>

In [ ]:
roslaunch ur5control endowrist_control.launch

## Pen Orientation operation

## Pen Position operation

When this specific push buttom is pressed, the robot arm has to change the TCP POSE in x direction a delta(x) value

Teh TCP frame is the mobile frame 

## Pen Open operation